In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os

In [ ]:
folder_name = "/Users/marcfvanzyl/Google Drive/ScienceFair/data/2019-11-08_Calibration_OF/"

In [ ]:
images = np.array([folder_name + f for f in os.listdir(folder_name) if f.endswith(".jpeg")])

In [ ]:
chessboard_num_squares_across = 12
chessboard_num_squares_up = 8
chessboard_square_size = 0.0613
chessboard_aruco_ratio = 0.7   # this is a fraction of chessboard_square_size
aruco_dict = aruco.Dictionary_get(aruco.DICT_5X5_250)

In [ ]:
board = aruco.CharucoBoard_create(chessboard_num_squares_across, 
                                  chessboard_num_squares_up, 
                                  chessboard_square_size, 
                                  chessboard_aruco_ratio*chessboard_square_size, 
                                  aruco_dict)

imboard = board.draw((12000, 8000))
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.imshow(imboard, cmap = plt.cm.gray, interpolation = "nearest")
ax.axis("off")
plt.show()

In [ ]:
image_set_number = 1

def plotImage(im, title = "Title"):

    fig, axs = plt.subplots(figsize=(20,8))
    axs.imshow(img)
    axs.set_title(title)
    plt.show()

In [ ]:
# set up list variables to store the results
allCorners = []
allIds = []
allObjectPoints = []
allImagePoints = []

# These are parameters used by the cv2.cornerSubPix function
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)

# get files by looping over the images array
for ind, image in enumerate(images):
    
    # image[0] is the L image
    filename = image
    img = cv2.imread(filename)
    
    
    

 
    # convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imsize = gray.shape
    # find the aruco corners and the ids of each corner
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict)
    
    print('Found {} aruco marker corners'.format(len(ids)))
    
    if len(ids)>0:
        (retval, checkerboardCorners,
         checkerboardIds) = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, board, )
        
        print('Found {} checker corners'.format(len(checkerboardIds)))
        if len(checkerboardIds)>0:
            # SUB PIXEL DETECTION
            for corner in checkerboardCorners:
                #print('Sub pixel optimization:')
                #print(corner)
                cv2.cornerSubPix(gray, corner,
                                 winSize = (5,5),
                                 zeroZone = (-1,-1),
                                 criteria = criteria)
                #print(corner)
                #print('+++'
    allCorners.append(checkerboardCorners)
    allIds.append(checkerboardIds)
    allObjectPoints.append(board.chessboardCorners[checkerboardIds.reshape(checkerboardIds.shape[0])])
    fig, axs = plt.subplots(figsize=(36,12))
    axs.imshow(cv2.aruco.drawDetectedCornersCharuco(img, checkerboardCorners, checkerboardIds))
    plt.show()
    
    

In [ ]:
print("CAMERA CALIBRATION")

cameraMatrixInit = np.array([[ 1000.,    0., imsize[0]/2.],
                             [    0., 1000., imsize[1]/2.],
                             [    0.,    0.,           1.]])

distCoeffsInit = np.zeros((5,1))
flags = (cv2.CALIB_USE_INTRINSIC_GUESS + cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
#flags = (cv2.CALIB_RATIONAL_MODEL)
(ret, camera_matrix, distortion_coefficients,
 rotation_vectors, translation_vectors,
 stdDeviationsIntrinsics, stdDeviationsExtrinsics,
 perViewErrors) = cv2.aruco.calibrateCameraCharucoExtended(
                  charucoCorners=allCorners,
                  charucoIds=allIds,
                  board=board,
                  imageSize=imsize,
                  cameraMatrix=cameraMatrixInit,
                  distCoeffs=distCoeffsInit,
                  flags=flags,
                  criteria=(cv2.TERM_CRITERIA_EPS & cv2.TERM_CRITERIA_COUNT, 1000000, 1e-11))

In [ ]:
i=12 # select image id

plt.figure(figsize=(18,18))
frame = cv2.imread(images[i])
img_undist = cv2.undistort(frame,camera_matrix,distortion_coefficients,None)
plt.subplot(1,2,1)
plt.imshow(frame)
plt.title("Raw image")
plt.axis("off")
plt.subplot(1,2,2)
plt.imshow(img_undist)
plt.title("Corrected image")
plt.axis("off")
plt.show()

In [ ]:
import pickle

camera_calibration = {'camera_name':'Optical Flow','ret':ret, 'mtx':camera_matrix, 'dist':distortion_coefficients , 'rvecs':rotation_vectors, 'tvecs':translation_vectors}
pickle.dump(camera_calibration, open('OFCameraCalibration.p', 'wb'))